In [1]:
import os
import sqlite3
from multiprocessing import freeze_support
from modin.db_conn import ModinDatabaseConnection
import modin.pandas as mpd
%load_ext autoreload
%autoreload 2
os.environ["MODIN_ENGINE"] = "ray"  # Modin will use Ray
def load_data_from_db(con):
    try:
        df = mpd.read_sql("SELECT * FROM data", con)
        return df
    except Exception as e:
        print(f"Error loading data: {e}")
        raise


freeze_support()
dbfile = '/home/tompouce/workspaces/mafat-challenge/train_data_for_competition/mini_training_set.db'

conn = ModinDatabaseConnection('sqlalchemy', f'sqlite:///{dbfile}')

# Can use get_connection to get underlying sqlalchemy engine
conn.get_connection()
db_df = load_data_from_db(conn)
print(db_df.head())

   Device_ID                   Datetime      URL  Domain_Name  Domain_cls1  \
0        124  2023-04-23 03:04:30+03:00     6466      2368671          755   
1        124  2023-04-23 03:04:30+03:00  2245864      1792903            0   
2        124  2023-04-23 03:04:30+03:00  1839478       107342          332   
3        124  2023-04-23 03:14:50+03:00  1172090       107342          332   
4        124  2023-04-23 03:14:50+03:00  1839478       107342          332   

   Domain_cls2  Domain_cls3  Domain_cls4  Target  
0          799            0            0       0  
1            0            0            0       0  
2            0            0            0       0  
3            0            0            0       0  
4            0            0            0       0  


In [2]:
# db_df.groupby("Device_ID").apply(lambda x: x["Datetime"].min())
print(db_df.head())


   Device_ID                   Datetime      URL  Domain_Name  Domain_cls1  \
0        124  2023-04-23 03:04:30+03:00     6466      2368671          755   
1        124  2023-04-23 03:04:30+03:00  2245864      1792903            0   
2        124  2023-04-23 03:04:30+03:00  1839478       107342          332   
3        124  2023-04-23 03:14:50+03:00  1172090       107342          332   
4        124  2023-04-23 03:14:50+03:00  1839478       107342          332   

   Domain_cls2  Domain_cls3  Domain_cls4  Target  
0          799            0            0       0  
1            0            0            0       0  
2            0            0            0       0  
3            0            0            0       0  
4            0            0            0       0  


In [3]:
# "23-04 to 18-05"
db_df["Datetime"] = mpd.to_datetime(db_df["Datetime"])
db_df["Datetime"]
# db_df.groupby("Device_ID").apply(lambda x: (x-x["Datetime"].min()).dt.days)

0          2023-04-23 03:04:30+03:00
1          2023-04-23 03:04:30+03:00
2          2023-04-23 03:04:30+03:00
3          2023-04-23 03:14:50+03:00
4          2023-04-23 03:14:50+03:00
                      ...           
32248978   2023-05-13 21:26:02+03:00
32248979   2023-05-13 21:26:03+03:00
32248980   2023-05-13 21:32:33+03:00
32248981   2023-05-13 21:32:38+03:00
32248982   2023-05-13 21:32:43+03:00
Name: Datetime, Length: 32248983, dtype: datetime64[ns, UTC+03:00]

In [4]:
domain_counts = db_df.groupby(["Device_ID","Domain_Name","Target"]).count()
domain_counts = domain_counts.reset_index()
domain_counts = domain_counts[["Device_ID","Domain_Name","Target","Datetime"]]
domain_counts.columns = ["source","target","Target","weight"]
domain_counts

,source,target,Target,weight
0,124,3930,0,4
1,124,4136,0,3
2,124,6450,0,3
3,124,12837,0,1
4,124,17665,0,2
...,...,...,...,...
288003,69967,2361028,1,4
288004,69967,2387835,1,34
288005,69967,2389761,1,72
288006,69967,2390487,1,1


In [5]:
len(db_df)

32248983

In [6]:
from node2vec import Node2Vec
import networkx as nx

In [7]:
del db_df

In [8]:
# url_list_per_domain = db_df.groupby("Domain_Name")["URL"].apply(lambda x: x["URL"].tolist())

In [9]:
# G = nx.from_pandas_edgelist(url_counts, "source", "target", create_using=nx.Graph(), edge_attr="weight")
# G.add_edges_from(zip(db_df["Domain"], db_df["URL"]))
# users=[]
# domains=[]
# websites=[]
# for device_id ,devide_data in db_df.groupby("Device_ID"):
    

In [10]:
domain_counts.weight

0           4
1           3
2           3
3           1
4           2
         ... 
288003      4
288004     34
288005     72
288006      1
288007    385
Name: weight, Length: 288008, dtype: int64

In [11]:

# urls_vs_label = url_counts.melt(id_vars=["source","weight","Target"], value_vars=["target"], var_name="url_type", value_name="count")

In [12]:
domain_counts

,source,target,Target,weight
0,124,3930,0,4
1,124,4136,0,3
2,124,6450,0,3
3,124,12837,0,1
4,124,17665,0,2
...,...,...,...,...
288003,69967,2361028,1,4
288004,69967,2387835,1,34
288005,69967,2389761,1,72
288006,69967,2390487,1,1


In [13]:
# url_corr = urls_vs_label[["Target","value"]].corr()
# url_corr

In [14]:
import numpy as np
# Get unique device IDs from url_counts
device_ids = domain_counts['source'].unique()
# Randomly shuffle the device IDs
np.random.seed(42)  # for reproducibility
np.random.shuffle(device_ids)

# Calculate split indices (80% train, 20% test)
split_idx = int(len(device_ids) * 0.8)

# Create train and test device ID sets
train_device_ids = device_ids[:split_idx]
test_device_ids = device_ids[split_idx:]

# Create mask for train/test split in url_counts
train_mask = domain_counts['source'].isin(train_device_ids)
test_mask = domain_counts['source'].isin(test_device_ids)

# Split url_counts into train and test sets
train_domain_counts = domain_counts[train_mask]
test_url_counts = domain_counts[test_mask]

print(f"Total devices: {len(device_ids)}")
print(f"Train devices: {len(train_device_ids)}")
print(f"Test devices: {len(test_device_ids)}")
print(f"\nTrain samples: {len(train_domain_counts)}")
print(f"Test samples: {len(test_url_counts)}")

Total devices: 615
Train devices: 492
Test devices: 123

Train samples: 228881
Test samples: 59127


In [15]:
pivot_matrix = train_domain_counts.pivot(index='source', columns='target', values='weight').fillna(0)pythonpythopythonn


In [16]:
pivot_matrix.shape

(492, 21743)

In [17]:
cov_mat = pivot_matrix.cov()

In [18]:
cov_mat.head()

2025-03-06 02:36:27,712 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 3.67 GiB -- Worker memory limit: 5.85 GiB
2025-03-06 03:34:06,395 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 4.09 GiB -- Worker memory limit: 5.85 GiB
2025-03-06 03:39:06,400 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- U

2025-03-06 03:45:18,205 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:35809 (pid=9865) exceeded 95% memory budget. Restarting...
2025-03-06 03:45:18,577 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:35809' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {'lambda-503d1af9af24073ab80a15c8e950471b', 'lambda-b071ad777dfe2e8131c2cbaeda1af533', 'lambda-445dbc9e2a6b60ab835cd7484fcd21f1', 'lambda-411ec65b8fb07156fb5e513fad99e914', 'lambda-837a09590191e862f9e7d1c7acdc1166', 'lambda-03bfd5bab96b4ef85b36da42f09b044e', 'lambda-f8cc9c78f7ce817e5a4124a7acb16886', 'lambda-cf9d7d476208b931c1eab7db8156a3e2', 'lambda-6d9f4e8f6f81a7491a9f9d81eccf4990', 'lambda-3e5452eaf32af32a2b649788cab4e35c', 'lambda-93514a9657632bda9218650f58f00699', 'lambda-7df8e21052e030772b5293017a0c2706', 'lambda-0c7193bdcaf2953fbf56a427a233ce88', 'lambda-8dea6edc81df0c0ea6a8cbc935f17c37', 'lambda-5b9c091d503eeacba8aec4ddd87de182', 'lambda-011d720ec4b7

FutureCancelledError: lambda-5943b6af519f31a1ae886176c2be6aa7 cancelled for reason: unknown.

FutureCancelledError: lambda-5943b6af519f31a1ae886176c2be6aa7 cancelled for reason: unknown.

In [19]:
chunk_size = 2000  # Adjust this value based on your available memory
n_cols = cov_mat.shape[1]
cov_path = "cov_matrices"

# Create directory if it doesn't exist
os.makedirs(cov_path, exist_ok=True)

for i in range(0, n_cols, chunk_size):
	end_idx = min(i + chunk_size, n_cols)
	chunk = cov_mat.iloc[:, i:end_idx]
	chunk_name = f'cov_mat_chunk_{i}_{end_idx}.parquet'
	chunk.to_parquet(
		os.path.join(cov_path, chunk_name),
		engine='pyarrow',
		compression='snappy'
	)
	print(f"Saved chunk {i} to {end_idx}")


FutureCancelledError: lambda-5943b6af519f31a1ae886176c2be6aa7 cancelled for reason: unknown.

In [ ]:
# Save covariance matrix in chunks to avoid memory issues
chunk_size = 5000  # Adjust this value based on your available memory
n_cols = cov_mat.shape[1]
cov_path = "cov_matrices"

# Create directory if it doesn't exist
os.makedirs(cov_path, exist_ok=True)

for i in range(0, n_cols, chunk_size):
	end_idx = min(i + chunk_size, n_cols)
	chunk = cov_mat.iloc[:, i:end_idx]
	chunk_name = f'cov_mat_chunk_{i}_{end_idx}.parquet'
	chunk.to_parquet(
		os.path.join(cov_path, chunk_name),
		engine='pyarrow',
		compression='snappy'
	)
	print(f"Saved chunk {i} to {end_idx}")


In [ ]:
print("keep in mind that the best resolution will be achieved with a resolution of urls/chain of urls, not domains")
print("Cluster url walks")
print("I want to cluster urls/url walks from a given domain, to 3 categories")
print("positive correlation, zero correlation, negative correlation, to_label")

In [ ]:
# import numpy as np
# # Convert to pivot table where rows are devices and columns are URLs

# # Calculate the covariance matrix
# # Calculate covariance matrix efficiently using numpy operations
# # Convert to numpy array for faster computation
# # matrix_dense = pivot_matrix.to_numpy()

# # # Center the data (subtract mean of each column)
# # matrix_centered = matrix_dense - np.mean(matrix_dense, axis=0)
# cov_mat = mpd.DataFrame()
# batch_size = 10000
# # Calculate covariance matrix in batches
# pivot_matrix
# for i in range(0, pivot_matrix.shape[1], batch_size):
#     # cov_mat = mpd.concat([cov_mat, pivot_matrix.iloc[:, i:i+batch_size].cov()])
#     # print(f"Processed {i} columns")     
#     out_path = os.path.join("/home/tompouce/workspaces/mafat-challenge/eda/cov_matrices",f"cov_matrix_{i}.parquet")
#     if os.path.exists(out_path):
#         print(f"Skipping {out_path}")
#         continue
#     # batch_end = min(i + batch_size, matrix_centered.shape[1])
#     # batch_cov = (np.dot(matrix_centered[:, i:batch_end].T, matrix_centered) / (matrix_centered.shape[0] - 1))
#     # batch_df = mpd.DataFrame(batch_cov, 
#     #                        index=pivot_matrix.columns[i:batch_end],
#     #                        columns=pivot_matrix.columns)
#     batch_df = pivot_matrix.iloc[:, i:i+batch_size].cov()
#     batch_df.to_parquet(out_path)
#     # print(f"Saved {out_path}")
#     # cov_mat = mpd.concat([cov_mat, batch_df])
    

In [ ]:
first_half = cov_mat.iloc[:,:cov_mat.shape[1]//2]
second_half = cov_mat.iloc[:,cov_mat.shape[1]//2:]

In [84]:
first_quarter = first_half.iloc[:first_half.shape[1]//100,:first_half.shape[1]//100]

In [ ]:
import ray.data as rd
import ray
# Convert Modin DataFrame to Ray Dataset
# First get the Pandas partitions from Modin
# Initialize Ray if not already initialized
if not ray.is_initialized():
    ray.init()

# Convert the large Modin DataFrame to Ray Dataset in batches
ds = rd.from_modin(cov_mat)

# Write the dataset to parquet files with automatic batching
ds.write_parquet(
    'url_cov_mat_rays',
    filesystem=None,  # Local filesystem
    row_group_size_bytes=100_000_000,  # Adjust based on available memory (100MB)
    compression='snappy'
)


In [ ]:
chunk_size = 100
n_cols = cov_mat.shape[1]
cov_path = "/home/tompouce/workspaces/mafat-challenge/eda/cov_matrices"
for i in range(0, n_cols, chunk_size):
    end_idx = min(i + chunk_size, n_cols)
    chunk = cov_mat.iloc[:, i:end_idx]
    chunk_name = f'url_cov_mat_chunk_{i}_{end_idx}.parquet'
    chunk.to_parquet(os.path.join(cov_path,chunk_name), engine='pyarrow', compression='snappy')

In [ ]:
# (cov_mat*100).astype("uint8").to_parquet('url_cov_mat.parquet', engine='pyarrow', compression='snappy')

In [ ]:
# N2V = Node2Vec(G, dimensions=64, walk_length=30, num_walks=200, workers=8)
# model = N2V.fit(window=10, min_count=1, batch_words=8)


In [ ]:
# model.wv.save_word2vec_format("user2url_node2vec_embeddings.emb")

In [ ]:
db_df.groupby("Domain_cls2")


In [ ]:
db_df.head()
divide_ids = list(db_df["Device_ID"].unique())
print(len(divide_ids))
regular_df = db_df[db_df["Device_ID"].isin(divide_ids[:20])]
# %debug

In [11]:
regular_df = regular_df._to_pandas()

In [ ]:
regular_df.columns

In [5]:
# db_df["count_weighted_by_days"] = db_df.groupby("Device_ID").apply(lambda x: (x["Datetime"]-x["Datetime"].min()).dt.days).astype("float32")
# db_df["count_weighted_by_days"] = db_df["count_weighted_by_days"] *0.1+1
# db_df.groupby("Device_ID").apply(lambda x: (x["count_weighted_by_days"]/x["count_weighted_by_days"].sum())).astype("float64")
# db_df.groupby("Device_ID").apply(lambda x: (x["count_weighted_by_days"]/x["count_weighted_by_days"].sum())).astype("float64")

# db_df.groupby("Device_ID")["count_weighted_by_days"].sum()